<a href="https://colab.research.google.com/github/starminalush/mlops_report/blob/main/ways_of_convert_rubert_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение

Этот ноутбук для бекендеров, которым дали модельку и сказали деплоить так, чтобы она быстро работала. И больше ничего не дали, кроме модельки
	
  (・_・ヾ

Устанавливаем нужные зависимости

In [1]:
!pip install onnx transformers onnxruntime-gpu==1.11.0 folium==0.2.1 optimum[onnxruntime]

     |████████████████████████████████| 12.8 MB 15.0 MB/s 
     |████████████████████████████████| 4.0 MB 48.8 MB/s 
     |████████████████████████████████| 108.9 MB 5.1 kB/s 
     |████████████████████████████████| 69 kB 4.4 MB/s 
     |████████████████████████████████| 66 kB 5.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 880 kB 54.4 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 57.6 MB/s 
     |████████████████████████████████| 6.6 MB 43.9 MB/s 
     |████████████████████████████████| 46 kB 5.0 MB/s 
     |████████████████████████████████| 5.2 MB 36.7 MB/s 
     |████████████████████████████████| 325 kB 58.3 MB/s 
     |████████████████████████████████| 136 kB 76.2 MB/s 
     |████████████████████████████████| 212 kB 69.1 MB/s 
     |████████████████████████████████| 1.1 MB 18.1 MB/s 
     |████

Фиксируем версии библиотек

In [2]:
!pip freeze > req.txt

Импорты

In [1]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from transformers.onnx import export
from pathlib import Path
from typing import Mapping, OrderedDict
from transformers.onnx import OnnxConfig
from transformers import AutoConfig
import onnxruntime as nxrun
import onnx
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from torch.nn.utils import prune
from optimum.onnxruntime import ORTQuantizer
from torch.onnx import TrainingMode
from onnxruntime.quantization import quantize_dynamic, QuantType

Качаем датасет, на котором будем проверять качество модели

In [2]:
!wget https://github.com/sismetanin/rureviews/raw/master/women-clothing-accessories.3-class.balanced.csv

--2022-05-03 17:54:40--  https://github.com/sismetanin/rureviews/raw/master/women-clothing-accessories.3-class.balanced.csv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sismetanin/rureviews/master/women-clothing-accessories.3-class.balanced.csv [following]
--2022-05-03 17:54:40--  https://raw.githubusercontent.com/sismetanin/rureviews/master/women-clothing-accessories.3-class.balanced.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21781685 (21M) [text/plain]
Saving to: ‘women-clothing-accessories.3-class.balanced.csv.1’

women-clothing-acce 100%[===================>]  20.77M  --.-KB/s   

# Об нейросети

В качестве подопытного будем использовать [rubert-base-cased-sentiment](https://huggingface.co/blanchefort/rubert-base-cased-sentiment) для классификации русских предложений. Данная нейросеть предсказывает 3 метки класса, в зависимости от тона предложения - позитивное, негативное или нейтральное

Запускаем нейросеть как есть

In [3]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

@torch.no_grad()
def predict(text, device):
    model_local = model.to(device)
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
    outputs = model_local(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).cpu().numpy()
    return predicted[0]

In [4]:
text = 'Как задолбали эти тупые правила: не есть кота, не бить посуду, не есть кота'

Проверим время инференса модели на gpu и cpu

In [10]:
%%timeit
predict(text, device = torch.device('cpu'))

1 loop, best of 5: 113 ms per loop


In [11]:
%%timeit
predict(text, device = torch.device('cuda'))

The slowest run took 1010.65 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 12.7 ms per loop


Проверим качество модели. Для проверки качества будем использовать один из датасетов, на котором обучалась модель, а именно [этот](https://github.com/sismetanin/rureviews)

In [5]:
df = pd.read_csv('/content/women-clothing-accessories.3-class.balanced.csv', delimiter='\t')
df.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


Для удобства немного изменим датасет - заменим метки класса на цифровые значения и выберем 1000 рандомных строк

In [6]:
df = df.sample(frac=1).reset_index(drop=True)
df = df[:1000]
mapping = {'negative': 2, 'positive': 1, 'neautral':0}
df = df.replace({'sentiment': mapping})
df.head()

,review,sentiment
0,"Очень хорошая куртка, довольно таки плотная. Я...",1
1,"Заказ так и не пришёл, деньги не вернули. Спор...",2
2,Пришло,0
3,"Заказывала S. Размер подошёл. Но есть одно но,...",0
4,на спине между лямками дырка. продавец ничего ...,0


Считаем качество

In [7]:
texts = list(df['review'])
labels = list(df['sentiment'])

In [8]:
predictions = [predict(text=t, device=torch.device('cuda')) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]
print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7570952595655819, recall: 0.7528932881996591, f1score: 0.7418477839235202


Сохраним оригинальную модель и посмотрим на ее вес

In [20]:
!mkdir output

mkdir: cannot create directory ‘output’: File exists


In [21]:
torch.save(model, 'output/original.pt')

In [22]:
!du -shc output/original.pt

679M	output/original.pt
679M	total


# ONNX

Формат Open Neural Network Exchange (ONNX) обеспечит общий способ представления данных, используемых в нейронных сетях. Большинство платформ имеют сегодня собственный специфический формат моделей, которые способны работать с моделями других платформ только при использовании специальных инструментов преобразования форматов.

ONNX позволит осуществлять свободный обмен информацией, которой обладают модели, без процедуры преобразования. Модель, обученную на одной платформе, можно будет использовать и на другой платформе. Также можно будет модель, обученную на одном фреймворке, перенести на другой фреймворк.

Перевести модель в ONNX можно несколькими способами:

1. Есть способ конвертации модели через torch.onnx

In [23]:
!mkdir -p output/onnx_transforms

In [26]:
#делаем dummy input
dummy_input0 = torch.randint(1, 224, (1,512))
dummy_input1 = torch.randint(0, 1, (1,512 ))
dummy_input2 =  torch.randint(0, 1, (1,512 ))
dummy_inputs = (dummy_input0,dummy_input1,dummy_input2)
device = torch.device('cpu')
with torch.no_grad():
  symbolic_names = {0:'batch_size', 1: 'max_seq_len'} 
  torch.onnx.export(model.to(device),               # модель, которую будем экспортировать
                    dummy_inputs,                         # input модели
                    "output/onnx_transforms/rubert-base-cased-sentiment_torch.onnx",   # путь сохранения модель
                    export_params=True,        
                    opset_version=11,          # версия ONNX, в который будем экспортировать модель
                    do_constant_folding=True,
                    input_names = ["input_ids","attention_mask","token_type_ids"],
                    output_names = ['output'],
                    dynamic_axes={'input_ids': symbolic_names,        #если у нас динамический размер input
                                  'attention_mask' : symbolic_names,
                                  'token_type_ids' : symbolic_names},
                    training=TrainingMode.EVAL
                    )

Пробуем запустить в ONNX на gpu и cpu и посмотреть время инференса

In [27]:
sess_options = nxrun.SessionOptions()
# запуск на gpu
providers = [
    'CUDAExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("output/onnx_transforms/rubert-base-cased-sentiment_torch.onnx", sess_options, providers)

In [9]:
def predict_onnx(text):
  inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='np')
  outputs  = model_ONNX.run(None, dict(inputs))[0][0]
  result = np.where(outputs == np.amax(outputs))[0][0]
  return result

In [29]:
%%timeit 
predict_onnx(text)

The slowest run took 13.51 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 5: 3.6 ms per loop


In [30]:
#запуск на cpu
providers = [
    'CPUExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("output/onnx_transforms/rubert-base-cased-sentiment_torch.onnx", sess_options, providers)

In [31]:
%%timeit 
predict_onnx(text)

10 loops, best of 5: 54.7 ms per loop


Считаем качество

In [32]:
predictions = [predict_onnx(text = t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.779944967747291, recall: 0.7718710037195792, f1score: 0.7701675746159881


Посмотрим на вес модели

In [33]:
!du -shc output/onnx_transforms/*

679M	output/onnx_transforms/rubert-base-cased-sentiment_torch.onnx
679M	total


Вывод:  по сравнению с оригинальной моделью скорость инференса модели стала на порядок выше, метрики качества не изменились

2. Есть библиотека transforms для трансформеров, [где все почти из коробки](https://huggingface.co/docs/transformers/serialization)

In [34]:
class DistilBertOnnxConfig(OnnxConfig):
    @property
    def inputs(self) -> Mapping[str, Mapping[int, str]]:
        return OrderedDict(
            [
                ("input_ids", {0: "batch", 1: "sequence"}),
                ("attention_mask", {0: "batch", 1: "sequence"}),
                ("token_type_ids", {0: "batch", 1: "sequence"}),
            ]
        )

In [35]:
config = AutoConfig.from_pretrained("blanchefort/rubert-base-cased-sentiment")
onnx_config_for_seq_clf = DistilBertOnnxConfig(config, task="sequence-classification")
print(onnx_config_for_seq_clf.outputs)

OrderedDict([('logits', {0: 'batch'})])


In [36]:
device = torch.device('cpu')
onnx_inputs, onnx_outputs = export(
        tokenizer,
        model.to(device),
        onnx_config_for_seq_clf,
        output=Path("output/onnx_transforms/rubert-base-cased-sentiment.onnx"),
        opset=11)

Если проверить скорость инференса и качество, получим то же самое, так что проверим только на gpu

In [37]:
providers = [
    'CUDAExecutionProvider'
]
model_ONNX = nxrun.InferenceSession("output/onnx_transforms/rubert-base-cased-sentiment.onnx", sess_options, providers)

In [38]:
%%timeit
predict_onnx(text)

100 loops, best of 5: 4.89 ms per loop


In [39]:
predictions = [predict_onnx(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.779944967747291, recall: 0.7718710037195792, f1score: 0.7701675746159881


# TorchScript

TorchScript — инструмент, который позволяет с помощью пары строк кода и нескольких щелчков мыши сделать из пайплайна на питоне отчуждаемое решение, которое можно встроить в систему на C++. А еще она будет на python работать быстрее из-за jit компиляции. В библиотеке transformers так же [есть почти из коробки](https://huggingface.co/docs/transformers/serialization#torchscript)

In [40]:
tokenizer_torchscript = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment', torchscript = True)
model_torchscript = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True, torchscript=True)

In [41]:
dummy_input0 = torch.randint(1, 224, (1,512))
dummy_input1 = torch.randint(0, 1, (1,512 ))
dummy_input2 =  torch.randint(0, 1, (1,512 ))
traced_model = torch.jit.trace(model_torchscript, [x.clone().detach() for x in dummy_inputs])

In [42]:
!mkdir -p output/torchscript

In [43]:
torch.jit.save(traced_model, "output/torchscript/rubert-base-cased-sentiment_traced.pt")

Пробуем загрузить и предиктить на gpu и cpu

In [44]:
traced_model = torch.jit.load("output/torchscript/rubert-base-cased-sentiment_traced.pt")

In [52]:
@torch.no_grad()
def predict_torchscript(text, device):
    local_model = traced_model.to(device)
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
    outputs = local_model(**inputs)[0]
    predicted = torch.nn.functional.softmax(outputs, dim=1)
    predicted = torch.argmax(predicted, dim=1).cpu().numpy()
    return predicted[0]

In [53]:
%%timeit
predict_torchscript(text = text, device = torch.device('cpu'))

The slowest run took 5.40 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 101 ms per loop


In [54]:
%%timeit
predict_torchscript(text = text, device = torch.device('cuda'))

The slowest run took 22.61 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 10 ms per loop


Считаем качество модели

In [55]:
predictions = [predict_torchscript(text=t, device = torch.device('cuda')) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]
print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.779944967747291, recall: 0.7718710037195792, f1score: 0.7701675746159881


 Посмотрим на вес модели

In [49]:
!du -shc output/torchscript/*

679M	output/torchscript/rubert-base-cased-sentiment_traced.pt
679M	total


Вывод:  по сравнению с оригинальной моделью скорость инференса модели стала немного выше, метрики качества не изменились. Лучше, чем ничего. А вообще надо а с++ запускать, чтобы увидеть адекватный результат

# Прунинг модели

Model Pruning — обрезание избыточных частей сети для ускорения инференса без потери точности. Наглядно — откуда, сколько и как можно вырезать.

Есть очень много способов прунинга моделей, но здесь мы рассмотрим способ прунинга attention слоев

Есть два варианта, как прунить модель.

1 вариант - делать через torch.nn.utils.prune. В качестве примера есть данный [ноутбук](https://github.com/Huffon/nlp-various-tutorials/blob/master/pruning-bert.ipynb)

2 вариант - библиотека [nn_pruning](https://github.com/huggingface/nn_pruning) от HuggingFace


[Ссылка](https://aclanthology.org/2020.repl4nlp-1.18.pdf) на почитать про прунинг модели BERT

Посмотрим на саму модель. Видим, что большую ее часть занимают attention слои. Давайте их и запруним и посмотрим, что получится

In [50]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [57]:
pruned_model = model

parameters_to_prune = ()
for i in range(12):
    parameters_to_prune += (
        (pruned_model.bert.encoder.layer[i].attention.self.key, 'weight'),
        (pruned_model.bert.encoder.layer[i].attention.self.query, 'weight'),
        (pruned_model.bert.encoder.layer[i].attention.self.value, 'weight'),
    )

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.8
)
for p in parameters_to_prune:
  prune.remove(p[0], 'weight')

Выведем, что получилось

In [58]:
for i in range(12):
    print(
        "Sparsity in Layer {}-th key weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(pruned_model.bert.encoder.layer[i].attention.self.key.weight == 0))
            / float(pruned_model.bert.encoder.layer[i].attention.self.key.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th query weightt: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(pruned_model.bert.encoder.layer[i].attention.self.query.weight == 0))
            / float(pruned_model.bert.encoder.layer[i].attention.self.query.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th value weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(pruned_model.bert.encoder.layer[i].attention.self.value.weight == 0))
            / float(pruned_model.bert.encoder.layer[i].attention.self.value.weight.nelement())
        )
    )
    print()

    
numerator, denominator = 0, 0
for i in range(12):
    numerator += torch.sum(pruned_model.bert.encoder.layer[i].attention.self.key.weight == 0)
    numerator += torch.sum(pruned_model.bert.encoder.layer[i].attention.self.query.weight == 0)
    numerator += torch.sum(pruned_model.bert.encoder.layer[i].attention.self.value.weight == 0)

    denominator += pruned_model.bert.encoder.layer[i].attention.self.key.weight.nelement()
    denominator += pruned_model.bert.encoder.layer[i].attention.self.query.weight.nelement()
    denominator += pruned_model.bert.encoder.layer[i].attention.self.value.weight.nelement()
    
print("Global sparsity: {:.2f}%".format(100. * float(numerator) / float(denominator)))

Sparsity in Layer 1-th key weight: 77.23%
Sparsity in Layer 1-th query weightt: 77.17%
Sparsity in Layer 1-th value weight: 91.60%

Sparsity in Layer 2-th key weight: 77.46%
Sparsity in Layer 2-th query weightt: 76.82%
Sparsity in Layer 2-th value weight: 90.37%

Sparsity in Layer 3-th key weight: 80.21%
Sparsity in Layer 3-th query weightt: 79.54%
Sparsity in Layer 3-th value weight: 87.01%

Sparsity in Layer 4-th key weight: 77.59%
Sparsity in Layer 4-th query weightt: 77.35%
Sparsity in Layer 4-th value weight: 88.18%

Sparsity in Layer 5-th key weight: 76.99%
Sparsity in Layer 5-th query weightt: 76.74%
Sparsity in Layer 5-th value weight: 85.86%

Sparsity in Layer 6-th key weight: 76.24%
Sparsity in Layer 6-th query weightt: 75.17%
Sparsity in Layer 6-th value weight: 82.86%

Sparsity in Layer 7-th key weight: 77.00%
Sparsity in Layer 7-th query weightt: 75.85%
Sparsity in Layer 7-th value weight: 83.59%

Sparsity in Layer 8-th key weight: 77.44%
Sparsity in Layer 8-th query weigh

Предиктим на запруненной модели на gpu и cpu

In [59]:
@torch.no_grad()
def predict_pruned(text, device):
    local_model = pruned_model.to(device)
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
    outputs = local_model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).cpu().numpy()
    return predicted[0]

In [60]:
%%timeit
predict_pruned(text, device = torch.device('cpu'))

10 loops, best of 5: 150 ms per loop


In [61]:
%%timeit
predict_pruned(text, device = torch.device('cuda'))

The slowest run took 16.14 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 21.6 ms per loop


In [62]:
!mkdir -p output/pruning

In [63]:
torch.save(pruned_model, 'output/pruning/rubert-base-cased-sentiment_pruned.pt')

Посчитаем качество модели

In [64]:
predictions = [predict_pruned(text = t, device=torch.device('cuda')) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]
print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.805674989730417, recall: 0.7873728838562007, f1score: 0.7917501120030411


 Посмотрим на вес модели

In [65]:
!du -shc output/pruning/*

679M	output/pruning/rubert-base-cased-sentiment_pruned.pt
679M	total


Вывод - вес модели не поменялся, качество даже чуть-чуть улучшилось по сравнению с оригинальной моделью, но не критично

# Квантизация

Квантизация означает уменьшение численной точности весов модели. Один из популярных методов — k-means квантизация. Имея веса модели в матрице W с десятичными числами, веса кластеризуются с помощью k-means в N кластеров. Затем матрица W трансформируется в матрицу целых чисел от 1 до N, каждое из которых является указателем к центру кластера. Так можно сжать каждый элемент изначальной матрицы из 32-битного десятичного числа в log(N)-битные целые числа.

Есть три вида квантизации - статическая, динамическая и Quantization-Aware-Training(QAT)

Динамическая квантизация не требует ничего, поэтому она самая простая

In [66]:
!mkdir -p output/quantization

Можно квантизировать модель через библиотеку onnxruntime 
(!Внимание, операция очень тяжелая, вам может не хватить памяти в колабе, лучше запускать отдельно. Так же onnxruntime не работает с квантизированными моделями на gpu, потому запустится она только на cpu)

In [10]:
model_fp32 = 'output/onnx_transforms/rubert-base-cased-sentiment_torch.onnx'
model_quant = 'output/quantization/rubert-base-cased-sentiment.quant.onnx'
quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type=QuantType.QUInt8)

Ignore MatMul due to non constant B: /[MatMul_68]
Ignore MatMul due to non constant B: /[MatMul_73]
Ignore MatMul due to non constant B: /[MatMul_162]
Ignore MatMul due to non constant B: /[MatMul_167]
Ignore MatMul due to non constant B: /[MatMul_256]
Ignore MatMul due to non constant B: /[MatMul_261]
Ignore MatMul due to non constant B: /[MatMul_350]
Ignore MatMul due to non constant B: /[MatMul_355]
Ignore MatMul due to non constant B: /[MatMul_444]
Ignore MatMul due to non constant B: /[MatMul_449]
Ignore MatMul due to non constant B: /[MatMul_538]
Ignore MatMul due to non constant B: /[MatMul_543]
Ignore MatMul due to non constant B: /[MatMul_632]
Ignore MatMul due to non constant B: /[MatMul_637]
Ignore MatMul due to non constant B: /[MatMul_726]
Ignore MatMul due to non constant B: /[MatMul_731]
Ignore MatMul due to non constant B: /[MatMul_820]
Ignore MatMul due to non constant B: /[MatMul_825]
Ignore MatMul due to non constant B: /[MatMul_914]
Ignore MatMul due to non constant

Пробуем запустить динамечески квантизированную ONNX модель и посмотреть на время инференса

In [12]:
sess_options = nxrun.SessionOptions()
providers = [
    'CPUExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("output/quantization/rubert-base-cased-sentiment.quant.onnx", sess_options, providers)

In [13]:
%%timeit
predict_onnx(text)

10 loops, best of 5: 37.9 ms per loop


Считаем качество

In [14]:
predictions = [predict_onnx(t) for t in texts]
precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7545168261175811, recall: 0.7505926047060564, f1score: 0.7370407334951347


Можно так же через библиотеку optimum от transformers

In [15]:
# The type of quantization to apply
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)
quantizer = ORTQuantizer.from_pretrained("blanchefort/rubert-base-cased-sentiment", feature="sequence-classification")

# Quantize the model!
quantizer.export(
    onnx_model_path="output/quantization/rubert-base-cased-sentiment.onnx",
    onnx_quantized_model_output_path="output/quantization/rubert-base-cased-sentiment_dyn_quantized.onnx",
    quantization_config=qconfig,
)

PosixPath('output/quantization/rubert-base-cased-sentiment_dyn_quantized.onnx')

Посмотрим на вес модели

In [16]:
!du -shc output/quantization/*

436M	output/quantization/rubert-base-cased-sentiment_dyn_quantized.onnx
679M	output/quantization/rubert-base-cased-sentiment.onnx
171M	output/quantization/rubert-base-cased-sentiment.quant.onnx
1.3G	total
